# Sports Savvy #

The goal of this product is to predict the performance of players in a given game. To accomplish this goal, our software will use past news and other available information and consider how similar conditions affected a player's past game. From there, it will calculate the probability of the player making his daily fantasy line.

In [33]:
import numpy as np
import pandas as pd
import requests

In [34]:
cd "C:\Users\IFE\OneDrive - eastern.edu\Documents\Fall 2023\CSCI-330B\NBA Players Stats"

[WinError 3] The system cannot find the path specified: 'C:\\Users\\IFE\\OneDrive - eastern.edu\\Documents\\Fall 2023\\CSCI-330B\\NBA Players Stats'
C:\Users\Indy.nelson\Documents\Sports Savvy SE project


Data preparations

In [35]:
player_stats_2019_2020 = pd.read_csv('2019-20_pbp.csv')
player_stats_2019_2020.head()

,URL,GameType,Location,Date,Time,WinningTeam,Quarter,SecLeft,AwayTeam,AwayPlay,...,EnterGame,LeaveGame,TurnoverPlayer,TurnoverType,TurnoverCause,TurnoverCauser,JumpballAwayPlayer,JumpballHomePlayer,JumpballPoss,Unnamed: 40
0,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,720,NOP,Jump ball: D. Favors vs. M. Gasol (L. Ball gai...,...,NaN,NaN,NaN,NaN,NaN,NaN,D. Favors - favorde01,M. Gasol - gasolma01,L. Ball - balllo01,NaN
1,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,708,NOP,L. Ball misses 2-pt jump shot from 11 ft,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,Offensive rebound by D. Favors,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,707,NOP,D. Favors makes 2-pt layup at rim,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,/boxscores/201910220TOR.html,regular,Scotiabank Arena Toronto Canada,October 22 2019,8:00 PM,TOR,1,689,NOP,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Dropping unnecessary columns and filling missing values with 0

In [36]:
# Columns to keep
keep_columns = ['Location', 'Date', 'WinningTeam', 'AwayTeam', 'HomeTeam', 'Shooter', 'ShotType', 'ShotOutcome', 'Assister', 'Rebounder', 'FreeThrowShooter', 'FreeThrowOutcome', 'TurnoverPlayer']

# Identify columns to drop
drop_columns = set(player_stats_2019_2020.columns) - set(keep_columns)

# Drop columns not in the specified set
player_stats_2019_2020.drop(columns = drop_columns, inplace = True)

player_stats_2019_2020.fillna(0, inplace = True)
player_stats_2019_2020

,Location,Date,WinningTeam,AwayTeam,HomeTeam,Shooter,ShotType,ShotOutcome,Assister,Rebounder,FreeThrowShooter,FreeThrowOutcome,TurnoverPlayer
0,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,0,0,0,0,0,0,0
1,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,L. Ball - balllo01,2-pt jump shot,miss,0,0,0,0,0
2,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,0,0,0,D. Favors - favorde01,0,0,0
3,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,D. Favors - favorde01,2-pt layup,make,0,0,0,0,0
4,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,O. Anunoby - anunoog01,2-pt layup,miss,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
539260,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,K. Olynyk - olynyke01,2-pt layup,make,0,0,0,0,0
539261,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,D. Howard - howardw01,3-pt jump shot,make,A. Caruso - carusal01,0,0,0,0
539262,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,S. Hill - hillso01,3-pt jump shot,make,0,0,0,0,0
539263,The Arena Bay Lake Florida,October 11 2020,LAL,LAL,MIA,0,0,0,0,0,0,0,0


Transforming *ShotType* column to numeric

In [37]:
player_stats_2019_2020['ShotType'].value_counts()

ShotType
0                 336868
3-pt jump shot     78344
2-pt layup         55362
2-pt jump shot     51077
2-pt dunk          11740
2-pt hook shot      5874
Name: count, dtype: int64

In [38]:
player_stats_2019_2020['ShotType'] = player_stats_2019_2020['ShotType'].str.replace(r'-pt jump shot|-pt layup|-pt dunk|-pt hook shot','', regex=True).astype(float)
player_stats_2019_2020 = player_stats_2019_2020[(player_stats_2019_2020['ShotOutcome'] != 'miss') & (player_stats_2019_2020['FreeThrowOutcome'] != 'miss')]
player_stats_2019_2020.head()

,Location,Date,WinningTeam,AwayTeam,HomeTeam,Shooter,ShotType,ShotOutcome,Assister,Rebounder,FreeThrowShooter,FreeThrowOutcome,TurnoverPlayer
0,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,0,0,0,0
2,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,D. Favors - favorde01,0,0,0
3,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,D. Favors - favorde01,2.0,make,0,0,0,0,0
5,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,J. Redick - redicjj01,0,0,0
7,Scotiabank Arena Toronto Canada,October 22 2019,TOR,NOP,TOR,0,NaN,0,0,F. VanVleet - vanvlfr01,0,0,0


Filling missing values in *ShotType* column with 0

In [39]:
player_stats_2019_2020['ShotType'].fillna(0, inplace = True)
player_stats_2019_2020['ShotType']

C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_16828\1666426506.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_2019_2020['ShotType'].fillna(0, inplace = True)


0         0.0
2         0.0
3         2.0
5         0.0
7         0.0
         ... 
539260    2.0
539261    3.0
539262    3.0
539263    0.0
539264    0.0
Name: ShotType, Length: 417979, dtype: float64

Aggregating the player stats to create the dataframe we will work with

In [40]:
player_stats_2019_2020['Date'] = pd.to_datetime(player_stats_2019_2020['Date'])
NBA2k19_pts_df = player_stats_2019_2020.groupby(['Shooter', 'Date','AwayTeam', 'HomeTeam'])['ShotType'].sum().reset_index()
NBA2k19_pts_df = NBA2k19_pts_df[NBA2k19_pts_df['Shooter'] != 0]


NBA2k19_pts_df['Shooter'] = NBA2k19_pts_df['Shooter'].str.split(' - ').str[0]
NBA2k19_pts_df

C:\Users\Indy.nelson\AppData\Local\Temp\ipykernel_16828\4111440781.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_stats_2019_2020['Date'] = pd.to_datetime(player_stats_2019_2020['Date'])


,Shooter,Date,AwayTeam,HomeTeam,ShotType
1143,A. Aminu,2019-10-23,CLE,ORL,3.0
1144,A. Aminu,2019-10-28,ORL,TOR,7.0
1145,A. Aminu,2019-10-30,NYK,ORL,8.0
1146,A. Aminu,2019-11-01,MIL,ORL,3.0
1147,A. Aminu,2019-11-05,ORL,OKC,8.0
...,...,...,...,...,...
22175,Z. Williamson,2020-07-30,UTA,NOP,12.0
22176,Z. Williamson,2020-08-01,NOP,LAC,6.0
22177,Z. Williamson,2020-08-03,MEM,NOP,18.0
22178,Z. Williamson,2020-08-06,NOP,SAC,20.0


Adding the other attributes we need

In [41]:
melted_df = pd.melt(player_stats_2019_2020[['Assister', 'Rebounder', 'TurnoverPlayer', 'FreeThrowShooter', 'Date']], id_vars='Date', value_vars=['Assister', 'Rebounder', 'TurnoverPlayer', 'FreeThrowShooter'], var_name='Role', value_name='Player')

# Group by 'date', 'player', and 'role', then calculate the count of occurrences for each group
NBA2k19_rebounds_df = melted_df.groupby(['Date', 'Player', 'Role']).size().unstack(fill_value=0).reset_index()
NBA2k19_rebounds_df = NBA2k19_rebounds_df[NBA2k19_rebounds_df['Player'] != 0]


NBA2k19_rebounds_df['Player'] = NBA2k19_rebounds_df['Player'].str.split(' - ').str[0]
NBA2k19_rebounds_df

Role,Date,Player,Assister,FreeThrowShooter,Rebounder,TurnoverPlayer
1,2019-10-22,A. Bradley,0,0,3,2
2,2019-10-22,A. Davis,5,9,9,3
3,2019-10-22,B. Ingram,5,4,5,2
4,2019-10-22,D. Favors,2,0,7,1
5,2019-10-22,D. Green,0,1,7,0
...,...,...,...,...,...,...
23487,2020-10-11,M. Morris,0,0,2,0
23488,2020-10-11,R. Rondo,4,0,4,4
23489,2020-10-11,S. Hill,1,0,0,0
23490,2020-10-11,T. Herro,4,0,3,4


In [42]:
NBA2k19 = pd.merge(NBA2k19_pts_df, NBA2k19_rebounds_df, left_on = ['Shooter', 'Date'], right_on = ['Player', 'Date'], how = 'outer')
NBA2k19['player_name'] = NBA2k19['Shooter'].combine_first(NBA2k19['Player'])
NBA2k19.drop(['Shooter', 'Player'], axis = 1, inplace = True)
NBA2k19 = NBA2k19[NBA2k19['player_name'] != 'Team']
NBA2k19.rename(columns = {'ShotType':'FG Points', 'Assister':'Assists', 'FreeThrowShooter':'FT Points', 'Rebounder':'Rebounds', 'TurnoverPlayer':'Turnovers',
                'player_name':'Player'}, inplace = True)
NBA2k19.fillna(0, inplace = True)
NBA2k19['Total Points'] = NBA2k19['FG Points'] + NBA2k19['FT Points']
NBA2k19 = NBA2k19[['Date', 'Player', 'Total Points', 'FG Points', 'FT Points', 'Rebounds', 'Assists', 'Turnovers']] # reorder columns
NBA2k19

,Date,Player,Total Points,FG Points,FT Points,Rebounds,Assists,Turnovers
0,2019-10-23,A. Aminu,5.0,3.0,2.0,7.0,0.0,1.0
1,2019-10-28,A. Aminu,7.0,7.0,0.0,5.0,3.0,1.0
2,2019-10-30,A. Aminu,9.0,8.0,1.0,8.0,1.0,1.0
3,2019-11-01,A. Aminu,3.0,3.0,0.0,5.0,1.0,2.0
4,2019-11-05,A. Aminu,13.0,8.0,5.0,5.0,1.0,2.0
...,...,...,...,...,...,...,...,...
24264,2020-10-06,D. Howard,0.0,0.0,0.0,2.0,0.0,2.0
24266,2020-10-09,A. Iguodala,0.0,0.0,0.0,6.0,2.0,1.0
24267,2020-10-09,D. Howard,2.0,0.0,2.0,2.0,0.0,1.0
24268,2020-10-09,M. Morris,0.0,0.0,0.0,1.0,0.0,3.0


We tried working with this dataset and merging it with our primary data to run our ML model but the difference between the name attributes made it quite complicated so after many tries and trying to use a different dsource of data, NBA_API we decided to go with ta simpler route create a randmized Injury List.

In [43]:
Injury10_20 = pd.read_csv('injuries_2010-2020.csv')
Injury10_20.head()

,Date,Team,Acquired,Relinquished,Notes
0,2010-10-03,Bulls,NaN,Carlos Boozer,fractured bone in right pinky finger (out inde...
1,2010-10-06,Pistons,NaN,Jonas Jerebko,torn right Achilles tendon (out indefinitely)
2,2010-10-06,Pistons,NaN,Terrico White,broken fifth metatarsal in right foot (out ind...
3,2010-10-08,Blazers,NaN,Jeff Ayres,torn ACL in right knee (out indefinitely)
4,2010-10-08,Nets,NaN,Troy Murphy,strained lower back (out indefinitely)


In [44]:
#readding home and away team from the first remade df
NBA2k19 = pd.merge(NBA2k19_pts_df, NBA2k19, left_on = ['Shooter', 'Date'], right_on = ['Player', 'Date'], how = 'left')

NBA2k19['IL List']=0
NBA2k19.head()

,Shooter,Date,AwayTeam,HomeTeam,ShotType,Player,Total Points,FG Points,FT Points,Rebounds,Assists,Turnovers,IL List
0,A. Aminu,2019-10-23,CLE,ORL,3.0,A. Aminu,5.0,3.0,2.0,7.0,0.0,1.0,0
1,A. Aminu,2019-10-28,ORL,TOR,7.0,A. Aminu,7.0,7.0,0.0,5.0,3.0,1.0,0
2,A. Aminu,2019-10-30,NYK,ORL,8.0,A. Aminu,9.0,8.0,1.0,8.0,1.0,1.0,0
3,A. Aminu,2019-11-01,MIL,ORL,3.0,A. Aminu,3.0,3.0,0.0,5.0,1.0,2.0,0
4,A. Aminu,2019-11-05,ORL,OKC,8.0,A. Aminu,13.0,8.0,5.0,5.0,1.0,2.0,0


In [45]:
NBA2k19.tail()

,Shooter,Date,AwayTeam,HomeTeam,ShotType,Player,Total Points,FG Points,FT Points,Rebounds,Assists,Turnovers,IL List
23279,Z. Williamson,2020-07-30,UTA,NOP,12.0,Z. Williamson,13.0,12.0,1.0,0.0,1.0,2.0,0
23280,Z. Williamson,2020-08-01,NOP,LAC,6.0,Z. Williamson,7.0,6.0,1.0,5.0,0.0,3.0,0
23281,Z. Williamson,2020-08-03,MEM,NOP,18.0,Z. Williamson,23.0,18.0,5.0,7.0,5.0,2.0,0
23282,Z. Williamson,2020-08-06,NOP,SAC,20.0,Z. Williamson,24.0,20.0,4.0,2.0,2.0,0.0,0
23283,Z. Williamson,2020-08-09,SAS,NOP,20.0,Z. Williamson,25.0,20.0,5.0,7.0,1.0,1.0,0


Choosing random games out of the games the players performed below their average as games were they were on Injury list.

In [46]:
# Step 2: Calculate average total stats for each player
avg_stats = NBA2k19.groupby('Player')[['Total Points','Rebounds','Assists']].mean().reset_index()
avg_stats['AvgTotalStats'] = avg_stats[['Total Points', 'Rebounds', 'Assists']].sum(axis=1)

# Step 3: Merge the average stats back into the original dataset
NBA2k19 = pd.merge(NBA2k19, avg_stats[['Player', 'AvgTotalStats']], on='Player', how='left')

# Step 4: Filter games where total stats are lower than the mean by 10 units
filtered_games = NBA2k19[NBA2k19['AvgTotalStats'] - 10 > NBA2k19['Total Points'] + NBA2k19['Rebounds'] + NBA2k19['Assists']]

# Step 5: Use a random algorithm to select some of those games
# Here, we use np.random.choice to randomly select 10% of the filtered games
randomly_selected_games = filtered_games.sample(frac=0.1, random_state=42)

# Step 6: Add 1 in the IL list column for the selected games
NBA2k19.loc[NBA2k19.set_index(['Date', 'AwayTeam', 'Player']).index.isin(randomly_selected_games.set_index(['Date', 'AwayTeam']).index), 'IL List'] = 1

# Display the resulting DataFrame
NBA2k19.tail()

,Shooter,Date,AwayTeam,HomeTeam,ShotType,Player,Total Points,FG Points,FT Points,Rebounds,Assists,Turnovers,IL List,AvgTotalStats
23279,Z. Williamson,2020-07-30,UTA,NOP,12.0,Z. Williamson,13.0,12.0,1.0,0.0,1.0,2.0,1,30.833333
23280,Z. Williamson,2020-08-01,NOP,LAC,6.0,Z. Williamson,7.0,6.0,1.0,5.0,0.0,3.0,0,30.833333
23281,Z. Williamson,2020-08-03,MEM,NOP,18.0,Z. Williamson,23.0,18.0,5.0,7.0,5.0,2.0,0,30.833333
23282,Z. Williamson,2020-08-06,NOP,SAC,20.0,Z. Williamson,24.0,20.0,4.0,2.0,2.0,0.0,0,30.833333
23283,Z. Williamson,2020-08-09,SAS,NOP,20.0,Z. Williamson,25.0,20.0,5.0,7.0,1.0,1.0,0,30.833333


In [47]:
#since it came with some etra nnwated columns lets drop them
NBA2k19.drop(columns = ['Shooter', 'ShotType','AvgTotalStats'], inplace = True)
NBA2k19.rename(columns = {'IL List':'IL_List', 'Total Points':'TotalPoints', 'FG Points':'FG_Points', 'FT Points':'FT_Points'}, inplace = True)
NBA2k19.head()

,Date,AwayTeam,HomeTeam,Player,TotalPoints,FG_Points,FT_Points,Rebounds,Assists,Turnovers,IL_List
0,2019-10-23,CLE,ORL,A. Aminu,5.0,3.0,2.0,7.0,0.0,1.0,0
1,2019-10-28,ORL,TOR,A. Aminu,7.0,7.0,0.0,5.0,3.0,1.0,0
2,2019-10-30,NYK,ORL,A. Aminu,9.0,8.0,1.0,8.0,1.0,1.0,0
3,2019-11-01,MIL,ORL,A. Aminu,3.0,3.0,0.0,5.0,1.0,2.0,0
4,2019-11-05,ORL,OKC,A. Aminu,13.0,8.0,5.0,5.0,1.0,2.0,0


Devlopping our ML Model

In [48]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder


# Encode categorical variables
le_player = LabelEncoder()
le_home_team = LabelEncoder()
le_away_team = LabelEncoder()

NBA2k19['Player_encoded'] = le_player.fit_transform(NBA2k19['Player'])
NBA2k19['HomeTeam_encoded'] = le_home_team.fit_transform(NBA2k19['HomeTeam'])
NBA2k19['AwayTeam_encoded'] = le_away_team.fit_transform(NBA2k19['AwayTeam'])

# Split into X and y for each target variable
X = NBA2k19[['Player_encoded', 'HomeTeam_encoded', 'AwayTeam_encoded', 'IL_List']]
y_fg = NBA2k19['FG_Points']
y_ft = NBA2k19['FT_Points']
y_rebs = NBA2k19['Rebounds']
y_asst = NBA2k19['Assists']
y_to = NBA2k19['Turnovers']

# Train-test split for each target variable
X_fg_train, X_fg_test, y_fg_train, y_fg_test = train_test_split(X, y_fg, test_size=0.2, random_state=42)
X_ft_train, X_ft_test, y_ft_train, y_ft_test = train_test_split(X, y_ft, test_size=0.2, random_state=42)
X_rebs_train, X_rebs_test, y_rebs_train, y_rebs_test = train_test_split(X, y_rebs, test_size=0.2, random_state=42)
X_asst_train, X_asst_test, y_asst_train, y_asst_test = train_test_split(X, y_asst, test_size=0.2, random_state=42)
X_to_train, X_to_test, y_to_train, y_to_test = train_test_split(X, y_to, test_size=0.2, random_state=42)

# Linear Regression Model for each target variable
model_fg = LinearRegression()
model_ft = LinearRegression()
model_rebs = LinearRegression()
model_asst = LinearRegression()
model_to = LinearRegression()

# Model Training for each target variable
model_fg.fit(X_fg_train, y_fg_train)
model_ft.fit(X_ft_train, y_ft_train)
model_rebs.fit(X_rebs_train, y_rebs_train)
model_asst.fit(X_asst_train, y_asst_train)
model_to.fit(X_to_train, y_to_train)

# Model Evaluation for each target variable
y_fg_pred = model_fg.predict(X_fg_test)
y_ft_pred = model_ft.predict(X_ft_test)
y_rebs_pred = model_rebs.predict(X_rebs_test)
y_asst_pred = model_asst.predict(X_asst_test)
y_to_pred = model_to.predict(X_to_test)

mse_fg = mean_squared_error(y_fg_test, y_fg_pred)
mse_ft = mean_squared_error(y_ft_test, y_ft_pred)
mse_rebs = mean_squared_error(y_rebs_test, y_rebs_pred)
mse_asst = mean_squared_error(y_asst_test, y_asst_pred)
mse_to = mean_squared_error(y_to_test, y_to_pred)

print(f'Mean Squared Error (FG_Points): {mse_fg}')
print(f'Mean Squared Error (FT_Points): {mse_ft}')
print(f'Mean Squared Error (Rebounds): {mse_rebs}')
print(f'Mean Squared Error (Assists): {mse_asst}')
print(f'Mean Squared Error (Turnovers): {mse_to}')


Mean Squared Error (FG_Points): 48.70757076303985
Mean Squared Error (FT_Points): 6.187253590927209
Mean Squared Error (Rebounds): 12.214648394194835
Mean Squared Error (Assists): 6.459467499410828
Mean Squared Error (Turnovers): 2.2173400235277216


In [18]:
pip install flask

Note: you may need to restart the kernel to use updated packages.


In [49]:
from flask import Flask, render_template, request

sport_savvy_app = Flask(__name__)

@sport_savvy_app.route('/')
def home():
    return render_template('index.html')

@sport_savvy_app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        # Get input values from the HTML form
        NewPlayer = request.form['player']
        NewHometeam = request.form['HomeTeam']
        NewAwayteam = request.form['AwayTeam']
        IL_List = int(request.form['IL_List'])

        # Encode the values
        player_encoded = le_player.transform([NewPlayer])[0]
        hometeam_encoded = le_home_team.transform([NewHometeam])[0]
        awayteam_encoded = le_away_team.transform([NewAwayteam])[0]

        # Create a DataFrame with the encoded values
        input_data = pd.DataFrame({
            'Player_encoded': [player_encoded],
            'HomeTeam_encoded': [hometeam_encoded],
            'AwayTeam_encoded': [awayteam_encoded],
            'IL_List': [IL_List],
        })

        # Make predictions using the machine learning model
        fg_pred = model_fg.predict(input_data)
        ft_pred = model_ft.predict(input_data)
        rebs_pred = model_rebs.predict(input_data)
        asst_pred = model_asst.predict(input_data)
        to_pred = model_to.predict(input_data)
        total_points_pred = fg_pred + ft_pred

        prediction = {
            'Total Points predicted': total_points_pred[0],
            'FG Points predicted': fg_pred[0],
            'FT Points predicted': ft_pred[0],
            'Rebounds predicted': rebs_pred[0],
            'Assists Predicted': asst_pred[0],
            'Turnovers Predicted': to_pred[0]
        }

        return render_template('result.html', prediction=prediction)

if __name__ == '__main__':
    import os
    port = int(os.environ.get('PORT', 5000))
    sport_savvy_app.run(host='0.0.0.0', port=port, debug=True, use_reloader=False)



 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.191.139.51:5000
Press CTRL+C to quit
127.0.0.1 - - [10/Dec/2023 23:08:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [10/Dec/2023 23:09:13] "POST /predict HTTP/1.1" 200 -


In [31]:
%tb

SystemExit: 1